# Research on Measuring Genome Containment
## Hannah Mannering 

Below is my script for creating logistic regression to predict, IDY, which is the known identity of genome containment between any two genomes. 

My script also evaluates if my equation performs better than MASH identity computation. 

In [1]:
# import necessary things
import pandas as pd
# read in output from our mash
data = pd.read_csv("data_testing_meta_NEW.csv")

In [2]:
# see what data is
data.head()

,REF,QRY,IDY,COV,MASHCOV,MASHFRAC,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,GCF_000273445,GCF_022070145,0.984479,10.000000,7.81944,0.720,NaN,NaN,NaN
1,GCF_000348705,GCF_022070145,0.989430,10.000000,7.95000,0.800,NaN,NaN,NaN
2,GCF_000448565,GCF_022070145,0.857890,10.000000,5.75000,0.040,NaN,NaN,NaN
3,GCF_000448605,GCF_022070145,0.830035,10.000000,6.00000,0.020,NaN,NaN,NaN
4,GCF_000448645,GCF_022070145,0.857890,10.000000,5.50000,0.040,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1676,GCF_900795145,GCF_009497975,0.783786,0.715267,1.30556,0.360,NaN,NaN,NaN
1677,GCF_900795155,GCF_009497975,0.789561,0.715267,1.32353,0.340,NaN,NaN,NaN
1678,GCF_900795165,GCF_009497975,0.783786,0.715267,1.35294,0.340,NaN,NaN,NaN
1679,GCF_900795245,GCF_009497975,0.789561,0.715267,1.36890,0.328,NaN,NaN,NaN


In [7]:
# drop unnecessary columns
data = data.drop(["Unnamed: 6","Unnamed: 7","Unnamed: 8"], axis=1)

In [11]:
# split dataset in features and target variable
X = data[['MASHCOV','MASHFRAC']]
y = data['IDY']

In [13]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)

/opt/conda/envs/python_CS703_f22/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [18]:
# import the class
from sklearn.linear_model import LinearRegression

# instantiate the model (using the default parameters)
model = LinearRegression()

# fit the model with data
model.fit(X_train, y_train)


LinearRegression()

In [19]:
# get the coefficient of determination
r_sq = model.score(X_train, y_train)
print(f"coefficient of determination: {r_sq}")

coefficient of determination: 0.7066728917623899


In [24]:
# get the model coef
model.coef_

array([-0.00167231,  0.24691634])

In [20]:
# print intercept
print(f"intercept: {model.intercept_}")

intercept: 0.8419985756134027


In [21]:
# print the slope
print(f"slope: {model.coef_}")

slope: [-0.00167231  0.24691634]


In [27]:
# Retrieve coefficients and intercept
coef = model.coef_[0]
intercept = model.intercept_

# Print linear regression equation
print(f'y = {coef:.5f}x + {intercept:.2f}')

y = -0.00167x + 0.84


In [50]:
# see what the predicted values are 
print(y_pred)

[1.00670178 1.02623675 0.84225942 ... 0.92368759 0.9206979  0.92383514]


In [52]:
# get metrics to evaluate performance
from sklearn.metrics import r2_score
# Predict output values for testing data
y_pred = model.predict(X_train)

# Evaluate performance of the model on testing data
r2 = r2_score(y_train, y_pred)
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
print(f'R-squared score: {r2:.2f}')
print(f'Mean squared error: {mse:.2f}')
print(f'Root mean squared error: {rmse:.2f}')

R-squared score: 0.71
Mean squared error: 0.00
Root mean squared error: 0.04


In [73]:
# add to this the predicted that is the equation from MASH paper
X_train['MASH_CALC'] = X_train['MASHFRAC'].apply(lambda x: x ** (1/100))


In [84]:
# add predictions to original df
df = X_train.assign(MYCALC=y_pred)
df = df.assign(IDY=data['IDY'])
df = df.assign(COVERAGE = data['COV'])
df['Difference_MashCalc'] = abs(df['IDY'] - df['new_column'])
df['Difference_MyCalc'] = abs(df['IDY'] - df['C'])
df.to_csv("Model_Results.csv")

# hypothesis : 
this model performs better 
returns values closer to the "truth"
when dealing with samples that have low coverage, compared to the usual mash formula 
which doesn't consider coverage at all